In [ ]:
import tkinter as tk
import matplotlib as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import numpy as np
from scipy import signal
import math

# 绘图函数
def drawPic():
    global r, g, b

    # 清空图像，以使得前后两次绘制的图像不会重叠
    drawPic.f.clf()
    drawPic.a = drawPic.f.add_subplot(111)
    drawPic.a.grid(color='k', linestyle='-.')
    Qa,Qb=IIR(rs,wn)


    # 每次绘图改变线条颜色用以区别
    color = ['b', 'r', 'y', 'g', 'grey', 'coral', 'darkgreen', 'c', 'cyan', 'steelblue']
    drawPic.a.set_xscale('log') 
    drawPic.a.plot(PositionalXaxis, PositionalYaxis, color=color[np.random.randint(len(color))])  # 绘制图形
    drawPic.a.set_xlabel('Frequency [radians / second]')
    drawPic.a.set_ylabel('Amplitude [dB]')
    drawPic.a.set_title('Chebyshev Type II frequency response ')
    
    #drawPic.a.plot(wn, ls="-",color = 'b') # cutoff frequencyls
    #drawPic.a.plot(-rs, ls=":",color = 'b') # rs
    drawPic.canvas.draw()
    #每次绘图完毕清空x，y
    PositionalXaxis.clear()
    PositionalYaxis.clear()
    return Qa, Qb
    

def IIR(rs, wn):
    global PositionalXaxis, PositionalYaxis, fig_num
    global xa, xb
    N=8
    fs=10*math.pow(10,6)
    #xb, xa = signal.cheby2(N, rs, wn, 'low', fs)
    xb, xa = signal.cheby2(N, rs, wn, 'low',fs=10*math.pow(10,6), output='ba')
    #a=a.tolist()
    #b=b.tolist()
    #w, h = signal.freqs(xb, xa)
    w, h = signal.freqz(xb, xa,fs=10*math.pow(10,6))
    #w=w.tolist()
    #h=h.tolist()
    PositionalYaxis=20 * np.log10(abs(h))
    PositionalXaxis=w
    PositionalYaxis=PositionalYaxis.tolist()
    PositionalXaxis=PositionalXaxis.tolist()
    print("a=%s" %xa)
    print("b=%s" %xb)
    Qa = np.int16(xa/np.max(abs(xa)) * 2**15 - 1)
    Qb = np.int16(xb/np.max(abs(xb)) * 2**15 - 1)
    Qa = Qa.tolist()
    Qb = Qb.tolist()
    #Qa转换
    Qa1=0
    Qa2=Qa
    Qa32=[0,0,0,0,0]
    print(Qa2)
    for i in range(len(Qa)):
        if Qa[i]>=0:
            Qa1=bin(Qa[i]).replace('0b','')
            #print(Qa1)
            Qa2[i]=Qa1.zfill(16)
        else:
            Qa2[i]=bin(Qa[i]&0xffff).replace('0b','')
    Qa32[0]=Qa2[0]+Qa2[1]
    Qa32[1]=Qa2[2]+Qa2[3]
    Qa32[2]=Qa2[4]+Qa2[5]
    Qa32[3]=Qa2[6]+Qa2[7]
    Qa32[4]=Qa2[8]+'0000000000000000'

    #Qb转换
    Qb1=0
    Qb2=Qb
    Qb32=[0,0,0,0,0]
    print(Qb2)
    for i in range(len(Qb)):
        if Qb[i]>=0:
            Qb1=bin(Qb[i]).replace('0b','')
            Qb2[i]=Qb1.zfill(16)
        else:
            Qb2[i]=bin(Qb[i]&0xffff).replace('0b','')
    Qb32[0]=Qb2[0]+Qb2[1]
    Qb32[1]=Qb2[2]+Qb2[3]
    Qb32[2]=Qb2[4]+Qb2[5]
    Qb32[3]=Qb2[6]+Qb2[7]
    Qb32[4]=Qb2[8]+'0000000000000000'

    print("Qa2=%s" %Qa2)
    print("Qa32=%s" %Qa32)
    print("Qb2=%s" %Qb2)
    print("Qb32=%s" %Qb32)

    
    return Qa, Qb


#   改变rs
def rs_enlarge():
    global rs, wn
    rs += 10
    var_rs.set('%.2f'%rs)
    Qa,Qb=drawPic()
    var_Qa.set(Qa)
    var_Qb.set(Qb)
    return rs, Qa, Qb
def rs_reduce():
    global rs
    rs -= 10
    if rs<=0:
        rs=0
    var_rs.set('%.2f'%rs)
    Qa,Qb=drawPic()
    var_Qa.set(Qa)
    var_Qb.set(Qb)
    return rs, Qa, Qb

#   改变wn
def wn_enlarge():
    global wn
    wn += 100000
    if wn>=5000000:
        wn=5000000
    var_wn.set('%.2f'%wn)
    Qa,Qb=drawPic()
    var_Qa.set(Qa)
    var_Qb.set(Qb)
    return wn, Qa, Qb
def wn_reduce():
    global wn
    wn -= 100000
    if wn<=0:
        wn=0
    var_wn.set('%.2f'%wn)
    Qa,Qb=drawPic()
    var_Qa.set(Qa)
    var_Qb.set(Qb)
    return wn, Qa, Qb

#   清空图像
def clear_pic():
    global r, g, b
    drawPic.f.clf()
    drawPic.canvas.draw()
    drawPic.a = drawPic.f.add_subplot(111)
    r = 0.6;
    g = 0.8;
    b = 0.8

#   重置
def reset():
    global rs, wn, Kd, r, g, b
    drawPic.f.clf()
    drawPic.canvas.draw()
    rs=60
    wn=3000000
    
    var_rs.set('%.2f' % rs)
    var_wn.set('%.2f' % wn)
    
    r = 0.6;
    g = 0.8;
    b = 0.8
    Qa,Qb=drawPic()
    var_Qa.set(Qa)
    var_Qb.set(Qb)
    return Qa, Qb

if __name__ == '__main__':

    # 实例化object，建立窗口window
    window = tk.Tk()
    # 给窗口的可视化起名字
    window.title('IIR_GUI')
    # 设定窗口的大小(长 * 宽)
    window.geometry('800x700')  # 这里的乘是小x
    plt.use('TkAgg')

    var_rs = tk.StringVar()    # 将label标签的内容设置为字符类型，用var来接收函数的传出内容用以显示在标签上
    var_wn = tk.StringVar()
    var_Qa = tk.IntVar()
    var_Qb = tk.IntVar()

    rs=60
    wn=3000000

    var_rs.set('%.2f' % rs)
    var_wn.set('%.2f' % wn)
    #PositionalXaxis = [0]
    #PositionalYaxis = [0]
    global PositionalXaxis, PositionalYaxis
    #PositionalXaxis.clear()
    #PositionalYaxis.clear()

    #定义颜色
    r=0.6; g=0.8; b=0.8

    # 绘图区域
    pic_area = tk.Label(window, bg='grey')
    pic_area.place(x=0, y=0, width='800', height='600')
    # 在Tk的GUI上放置一个画布
    drawPic.f = Figure(figsize=(5, 4), dpi=100)
    drawPic.canvas = FigureCanvasTkAgg(drawPic.f, master=pic_area)
    drawPic.canvas.draw()
    drawPic.canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)


    #调节控件
    rs_label = tk.Label(textvariable=var_rs, bg='grey')
    rs_label.place(x=20, y= 630, width='50', height='50')
    rs_add_Btn = tk.Button(window, text='Rs Enlarge', font=('Arial', 10), width=10, height=1, command=rs_enlarge)
    rs_add_Btn.place(x=80, y=620, width='100', height='30')
    rs_down_Btn = tk.Button(window,text='Rs Reduce', font=('Arial', 10), width=10, height=1, command=rs_reduce)
    rs_down_Btn.place(x=80, y=660, width='100', height='30')

    Ki_label = tk.Label(textvariable=var_wn, bg='grey')
    Ki_label.place(x=300, y= 630, width='100', height='50')
    Ki_add_Btn = tk.Button(window, text='Wn Enlarge', font=('Arial', 10), width=10, height=1, command=wn_enlarge)
    Ki_add_Btn.place(x=410, y=620, width='100', height='30')
    Ki_down_Btn = tk.Button(window,text='Wn Reduce', font=('Arial', 10), width=10, height=1, command=wn_reduce)
    Ki_down_Btn.place(x=410, y=660, width='100', height='30')


    Clear_Btn = tk.Button(window, text='Clear', font=('Arial', 10), width=10, height=1, command=clear_pic)
    Clear_Btn.place(x=580, y=620, width='100', height='70')

    Reset_Btn = tk.Button(window, text='Reset', font=('Arial', 10), width=10, height=1, command=reset)
    Reset_Btn.place(x=680, y=620, width='100', height='70')
    
    #print("IIR_coeff_Qa=%s" %var_Qa)
    
    window.mainloop()
   
    


a=[1.00000000e+00 2.17695390e-01 1.29761445e+00 3.46752111e-01
 5.05336441e-01 1.27540174e-01 6.25239329e-02 1.00423782e-02
 1.22982023e-03]
b=[0.03403411 0.16827386 0.43206473 0.72336864 0.853252   0.72336864
 0.43206473 0.16827386 0.03403411]
[25251, 5496, 32767, 8755, 12760, 3219, 1577, 252, 30]
[1306, 6461, 16591, 27779, 32767, 27779, 16591, 6461, 1306]
Qa2=['0110001010100011', '0001010101111000', '0111111111111111', '0010001000110011', '0011000111011000', '0000110010010011', '0000011000101001', '0000000011111100', '0000000000011110']
Qa32=['01100010101000110001010101111000', '01111111111111110010001000110011', '00110001110110000000110010010011', '00000110001010010000000011111100', '00000000000111100000000000000000']
Qb2=['0000010100011010', '0001100100111101', '0100000011001111', '0110110010000011', '0111111111111111', '0110110010000011', '0100000011001111', '0001100100111101', '0000010100011010']
Qb32=['00000101000110100001100100111101', '01000000110011110110110010000011', '01111